# Assignment 2
## Group name: [enter your group name with either the prefix "ID2214-" or "FID3214-"]
### Project members: 
Zuzanna Gorczyca, zgo@kth.se

Alga Nour Elimane, nealga@kth.se

Tse An Shih, tashih@kth.se


### Declaration:
By submitting this assignment, it is hereby declared that all group members listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214), no part of the solution has been provided by someone not listed as a project member above, and no part of the solution has been generated by a system.

It is furthermore declared that the submitted assignment will not be shared during the course, with any individual other than the group members listed above and teachers of the course ID2214/FID3214. In particular, the assignment will not be uploaded to any public repository. The submitted assignment can be shared after the course only if written consent has been provided by the course responsible of ID2214/FID3214.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy and pandas may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas and time

In [5]:
import numpy as np
import pandas as pd
import time

In [6]:
from platform import python_version

print(f"Python version: {python_version()}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

Python version: 3.13.5
NumPy version: 2.1.3
Pandas version: 2.2.3


## Reused functions from Assignment 1

In [85]:
#COLUMN FILTER 
def create_column_filter(df):
    df_copy=df.copy()
    col_keep=["CLASS","ID"]

    for col in df_copy.columns:
        if col not in col_keep: 
            if df_copy[col].dropna().nunique()<=1:
                df_copy=df_copy.drop(columns=[col])

    return df_copy,df_copy.columns.tolist()

def apply_column_filter(df,column_filter):
    df_copy=df.copy()
    df_copy=df_copy[column_filter]
    return df_copy
#==============================================================================================

#NORMALIZATION MAPING
def create_normalization(df,normalizationtype):
    df_copy=df.copy()
    normalization={}
    col_keep=["CLASS","ID"]

    numeric_cols=df_copy.select_dtypes(include=["int64","float64"]).columns
    numeric_cols=[c for c in numeric_cols if c not in col_keep]
    
    if normalizationtype=="minmax":
        min_value=df_copy[numeric_cols].min()
        max_value=df_copy[numeric_cols].max()
        df_copy[numeric_cols]=(df_copy[numeric_cols]-min_value)/(max_value-min_value)
        for c in numeric_cols:
            normalization[c]=("minmax",min_value[c],max_value[c])
    elif normalizationtype=="zscore":
        mean_value=df_copy[numeric_cols].mean()
        std_value=df_copy[numeric_cols].std()
        df_copy[numeric_cols]=(df_copy[numeric_cols]-mean_value)/std_value
        for c in numeric_cols:
            normalization[c]=("zscore",mean_value[c],std_value[c])
    else:
        print("Use minmax or zscore")
        normalization=None
    
    return df_copy,normalization

def apply_normalization(df,normalization):
    df_copy=df.copy()
    for col in normalization:
        normalization_type,value_1,value_2=normalization[col] 
        if normalization_type=="minmax":
            df_copy[col]=(df_copy[col]-value_1)/(value_2-value_1)
            df_copy[col]=df_copy[col].clip(0, 1)

        elif normalization_type=="zscore":
            df_copy[col]=(df_copy[col]-value_1)/value_2
    
    return df_copy

#==============================================================================================

#INPUTATION

# Insert the functions create_imputation and apply_imputation below (after the comments)
#
# Input to create_imputation:
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
#
# Output from create_imputation:
# df         - a new dataframe, where each missing numeric value in a column has been replaced by the mean of that column 
#              and each missing categoric value in a column has been replaced by the mode of that column
# imputation - a mapping (dictionary) from column name to value that has replaced missing values
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
#
# Hint 2: Handle columns of type "float" or "int" only (and which are not labeled "CLASS" or "ID") in one way
#         and columns of type "object" and "category" in other ways
#
# Hint 3: Consider using the pandas functions mean and mode respectively, as well as fillna
#
# Hint 4: In the rare case of all values in a column being missing*, replace numeric values with 0,
#         object values with "" and category values with the first category (cat.categories[0])  
#
#         *Note that this will not occur if the previous column filter function has been applied
#
# Input to apply_imputation:
# df         - a dataframe
# imputation - a mapping (dictionary) from column name to value that should replace missing values
#
# Output from apply_imputation:
# df - a new dataframe, where each missing value has been replaced according to the mapping
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
#
# Hint 2: Consider using fillna

#create_imputation
def create_imputation(df):
    df_copy=df.copy()
    col_keep=["CLASS","ID"]
    imputation={}
    
    cols=df_copy.select_dtypes(include=["int64","float64", "category", "object"]).columns
    cols=[n for n in cols if n not in col_keep]
    for col in cols:
        if df_copy[col].dtype in ["int64", "float64"]:
            if pd.isna(df_copy[col]).all():
                df_copy[col]=0
                imputation[col]=0
            else:
                mean_val = df_copy[col].mean()
                df_copy[col]=df_copy[col].fillna(mean_val)
                imputation[col]=mean_val

        elif df_copy[col].dtype == "O":
            if pd.isna(df_copy[col]).all():
                df_copy[col]=""
                imputation[col]=""
            else:
                mode_val = df_copy[col].mode().iloc[0]
                df_copy[col]=df_copy[col].fillna(mode_val)
                imputation[col]=mode_val

        elif df_copy[col].dtype == "category":
            if pd.isna(df_copy[col]).all():
                cat_val = df_copy[col].cat.categories[0]
                df_copy[col]=df_copy[col].fillna(cat_val)
                imputation[col]=cat_val
            else:
                mode_val = df_copy[col].mode().iloc[0]
                df_copy[col]=df_copy[col].fillna(mode_val)
                imputation[col]=mode_val
    
    return df_copy,imputation

#apply_imputation
def apply_imputation(df,imputation):
    df_copy=df.copy()
    for col in imputation:
        fill_value = imputation[col]  # directly use the value
        df_copy[col]=df_copy[col].fillna(fill_value)
    return df_copy


#==============================================================================================


#ONE-HOT MAPPING 

def create_one_hot(df):
    df_copy = df.copy()
    one_hot = {}
    category_cols = df_copy.select_dtypes(include=['object', 'category']).columns.tolist()
    category_cols_no_ids = [c for c in category_cols if c not in ["ID", "CLASS"]]
    for column in category_cols_no_ids: 
        all_cat = set(cat for cat in df_copy[column])
        one_hot[column] = {cat: "" for cat in all_cat}
        for cat in all_cat:
            df_copy[f'{column} {cat}'] = df_copy[column].apply(lambda x: 1.0 if cat in x else 0.0) 
            one_hot[column][cat] = f'{column} {cat}'
        df_copy = df_copy.drop(column, axis=1)

    return df_copy, one_hot


def apply_one_hot(df, one_hot):
    df_copy = df.copy()
    category_cols = df_copy.select_dtypes(include=['object', 'category']).columns.tolist()
    category_cols_no_ids = [c for c in category_cols if c not in ["ID", "CLASS"]]
    for kay, value in one_hot.items():
        for cat, new_name in value.items():
            df_copy[new_name] = df_copy[kay].apply(lambda x: 1.0 if cat in x else 0.0)
        df_copy = df_copy.drop(kay, axis=1)
    
    return df_copy

#==============================================================================================

#DISCRETIZATION MAPING
def create_bins(df, nobins, bintype="equal-width"):
    df_tmp = df.drop(["CLASS","ID"], axis=1, errors="ignore").select_dtypes(include=["int","float"])

    binning = {}
    df_out = pd.DataFrame()
    for col in df_tmp.columns:
        if (bintype == "equal-width"):
            res_tmp, bins_tmp = pd.cut(df_tmp[col], nobins, retbins=True, labels=False, duplicates="drop")
        elif (bintype == "equal-size"):
            res_tmp, bins_tmp = pd.qcut(df_tmp[col], nobins, retbins=True, labels=False, duplicates="drop")
            
        bins_tmp[0] = -np.inf # Set the first element of the binning list to -np.inf
        bins_tmp[-1] = np.inf # Set the last element of the binning list to np.inf
        
        binning[col] = tuple(bins_tmp)
        
    df_out = apply_bins(df,binning) # Apply binning
    return df_out,binning


def apply_bins(df,binning):
    df_out = df.copy()
    for col in binning:
        df_out[col] = pd.DataFrame(pd.cut(df_out[col],binning[col],labels=False, duplicates="drop")).astype("category")
    return df_out

#==============================================================================================
def accuracy(df, correctlabels):
    # Input to accuracy:
    # df            - a dataframe with class labels as column names and each row corresponding to
    #                 a prediction with estimated probabilities for each class
    # correctlabels - an array (or list) of the correct class label for each prediction
    #                 (the number of correct labels must equal the number of rows in df)
    
    # Output from accuracy:
    # accuracy - the fraction of cases for which the predicted class label coincides with the correct label
    

    # Hint: In case the label receiving the highest probability is not unique, you may
    #       resolve that by picking the first (as ordered by the column names) or 
    #       by randomly selecting one of the labels with highest probaility.
    df_copy = df.copy()
    predictions = df_copy.idxmax(axis=1).values
    sum = 0
    for i in range(len(predictions)):
        if predictions[i] == correctlabels[i]:
            sum += 1

    return sum/len(correctlabels)
def brier_score( df , correctlabels ):
    scores = []
    for i, label in enumerate(correctlabels):
        correct_vector = np.zeros(len(df.columns))
        index = np.where(df.columns == label)[0][0]
        #print(label, index, df.iloc[:, index].values)
        correct_vector[index] = 1
        row_score = (df.iloc[i].values - correct_vector)**2
        scores.append(row_score)
    return np.sum(scores)/len(df) 
def auc( df , correctlabels):
     #binary auc for each class label 
    classes = df.columns 
    total_auc = 0
    for c in classes:
        scores = {}   
        for index, score in enumerate(df[c].values): 
            if score not in scores:
                scores[score] = [0, 0]  # initialization
            
            if correctlabels[index] == c:  
                scores[score][0] += 1 # because we want [posi , neg ] 
            else: 
                scores[score][1] += 1  
    #hint n4 
        sorted_scores = [scores[s] for s in sorted(scores.keys(), reverse=True)]
        positives = [s[0] for s in sorted_scores]  
        negatives = [s[1] for s in sorted_scores]
        tot_pos = sum(positives)
        tot_neg = sum(negatives)
        cum_pos = 0
        auc_class = 0
       #hint 5 
        for i in range(len(positives)):
            if negatives[i] == 0:
                cum_pos += positives[i]  # only positives
            elif positives[i] == 0: # only negatives 
                auc_class += (cum_pos / tot_pos) * (negatives[i] / tot_neg)
            else:
                # both positives and negatives exist
                auc_class += (cum_pos / tot_pos) * (negatives[i] / tot_neg)
                auc_class += (positives[i] / tot_pos) * (negatives[i] / tot_neg) / 2
                cum_pos += positives[i]

        # hint 6 
        class_weight = sum(1 for label in correctlabels if label == c) / len(df)
        total_auc += class_weight * auc_class

    return total_auc

## 1. Define the class kNN

In [8]:
# Define the class kNN with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, normalization, one_hot, labels, training_labels, training_data, training_time

def __input__(self):
    self.column_filter=None
    self.imputation=None
    self.normalization=None
    self.one_hot=None
    self.labels=None
    self.training_labels=None
    self.training_data=None
    self.training_time=None

# Input to fit:
# self              - the object itself
# df                - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# normalizationtype - "minmax" (default) or "zscore"
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter   - a column filter (see Assignment 1) from df
# self.imputation      - an imputation mapping (see Assignment 1) from df
# self.normalization   - a normalization mapping (see Assignment 1), using normalizationtype from the imputed df
# self.one_hot         - a one-hot mapping (see Assignment 1)
# self.training_labels - a pandas series corresponding to the "CLASS" column, set to be of type "category" 
# self.labels          - a list of the categories (class labels) of the previous series
# self.training_data   - the values (an ndarray) of the transformed dataframe, i.e., after employing imputation, 
#                        normalization, and possibly one-hot encoding, and also after removing the "CLASS" and "ID" columns
#
# Note that the function does not return anything but just assigns values to the attributes of the object.

def fit(self,df,normalizationtype):
    df_copy=df.copy()

    #training_labels
    labels=df_copy["CLASS"]
    labels=labels.astype("category")
    self.training_labels=labels

    #labels
    self.labels=self.training_labels.cat.categories.tolist()    

    #drop "CLASS"
    df_keep=df_copy.drop(["CLASS","ID"],axis=1)
    
    #column_filter
    filter_df,column_filter=create_column_filter(df_keep)
    self.column_filter=column_filter
    df_keep=filter_df

    #imputation
    imputed_df,imputation=create_imputation(df_keep)
    self.imputation=imputation
    df_keep=imputed_df

    #normalization
    normalized_df,normalization=create_normalization(df_keep,normalizationtype)
    self.normalization=normalization
    df_keep=normalized_df

    #one_hot
    new_df,one_hot=create_one_hot(df_keep)
    self.one_hot=one_hot
    df_keep=new_df

    #training_data
    self.training_data=df_keep.to_numpy(dtype=float)



# Input to predict:
# self - the object itself
# df   - a dataframe
# k    - an integer >= 1 (default = 5)
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the relative class frequencies in the set of class labels from the k nearest 
#               (with respect to Euclidean distance) neighbors in training_data
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply column filtering, imputation, normalization and one-hot
#
# Hint 2: Get the numerical values (as an ndarray) from the resulting dataframe and iterate over the rows 
#         calling some sub-function, e.g., get_nearest_neighbor_predictions(x_test,k), which for a test row
#         (numerical input feature values) finds the k nearest neighbors and calculate the class probabilities.
#
# Hint 3: This sub-function may first find the distances to all training instances, e.g., pairs consisting of
#         training instance index and distance, and then sort them according to distance, and then (using the indexes
#         of the k closest instances) find the corresponding labels and calculate the relative class frequencies



In [9]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

knn_model = kNN()

t0 = time.perf_counter()
knn_model.fit(glass_train_df)
print("Training time: {0:.2f} s.".format(time.perf_counter()-t0))

test_labels = glass_test_df["CLASS"]

k_values = [1,3,5,7,9]
results = np.empty((len(k_values),3))

for i in range(len(k_values)):
    t0 = time.perf_counter()
    predictions = knn_model.predict(glass_test_df,k=k_values[i])
    print("Testing time (k={0}): {1:.2f} s.".format(k_values[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=k_values,columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

NameError: name 'kNN' is not defined

In [ ]:
train_labels = glass_train_df["CLASS"]
predictions = knn_model.predict(glass_train_df,k=1)
print("Accuracy on training set (k=1): {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set (k=1): {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set (k=1): {0:.4f}".format(brier_score(predictions,train_labels)))

### Comment on assumptions, things that do not work properly, etc.


## 2. Define the class NaiveBayes

In [86]:
# Define the class NaiveBayes with three functions __init__, fit and predict (after the comments):
# 
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, binning, labels, class_priors, feature_class_value_counts, feature_class_counts
#
# Input to fit:
# self    - the object itself
# df      - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins  - no. of bins (default = 10)
# bintype - either "equal-width" (default) or "equal-size" 
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter              - a column filter (see Assignment 1) from df
# self.binning                    - a discretization mapping (see Assignment 1) from df
# self.class_priors               - a mapping (dictionary) from the labels (categories) of the "CLASS" column of df,
#                                   to the relative frequencies of the labels
# self.labels                     - a list of the categories (class labels) of the "CLASS" column of df
# self.feature_class_value_counts - a mapping from the feature (column name) to the number of
#                                   training instances with a specific combination of (non-missing, categorical) 
#                                   value for the feature and class label
# self.feature_class_counts       - a mapping from the feature (column name) to the number of
#                                   training instances with a specific class label and some (non-missing, categorical) 
#                                   value for the feature
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the naive approximation of Bayes rule (see lecture slides)
#
# Hint 1: First apply the column filter and discretization
#
# Hint 2: Iterating over either columns or rows, and for each possible class label, calculate the relative
#         frequency of the observed feature value given the class (using feature_class_value_counts and 
#         feature_class_counts) 
#
# Hint 3: Calculate the non-normalized estimated class probabilities by multiplying the class priors to the
#         product of the relative frequencies
#
# Hint 4: Normalize the probabilities by dividing by the sum of the non-normalized probabilities; in case
#         this sum is zero, then set the probabilities to the class priors
#
# Hint 5: To clarify the assignment text a little: self.feature_class_value_counts should be a mapping from 
#         a column name (a specific feature) to another mapping, which given a class label and a value for 
#         the feature, returns the number of training instances which have included this combination, 
#         i.e., the number of training instances with both the specific class label and this value on the feature.
#
# Hint 6: As an additional hint, you may take a look at the slides from the NumPy and pandas lecture, to see how you 
#         may use "groupby" in combination with "size" to get the counts for combinations of values from two columns.

class NaiveBayes : 

    def __init__(self):  
        self.column_filter=None 
        self.binning=None 
        self.labels =None 
        self.class_priors = None 
        self.feature_class_value_counts = None 
        self.feature_class_counts = None 


    def fit(self , df , nobins=10 , bintype = "equal-width" ):

        #applying the filter of assignment 1 
        df_filtered, self.column_filter = create_column_filter(df)

        #bining 
        df_binned, self.binning = create_bins(df_filtered, nobins=nobins, bintype=bintype) 

        # we  allow "MISSING" in categorical columns 
        for col in df_binned.columns:
            if str(df_binned[col].dtype) == "category":
                df_binned[col] = df_binned[col].cat.add_categories(["MISSING"])
        df_binned = df_binned.fillna("MISSING")

        #labels :
        df_binned["CLASS"] = df_binned["CLASS"].astype("category")   
        self.labels = df_binned['CLASS'].cat.categories.tolist() 
        # self.class prior
        class_counts = df_binned['CLASS'].value_counts()
        self.class_priors = dict(class_counts / len(df_binned))

        #feautures : 
        #mapping 
        self.feature_class_value_counts = {}
        self.feature_class_counts = {}

        #we iterate on all colomns except Class and Id 
        features = [col for col in df_binned.columns if col not in ['CLASS', 'ID']]
        
        for feature in features:
            df_drop = df_binned[[feature, 'CLASS']].dropna()
        
            counts = df_drop.groupby([feature, 'CLASS'], observed=False).size()
            self.feature_class_value_counts[feature] = counts.to_dict()
        
            class_totals = df_drop.groupby('CLASS', observed=False).size()
            self.feature_class_counts[feature] = class_totals.to_dict()


    def predict(self , df):

        #hint1 : filter and discretization :
        df_filtered = apply_column_filter(df, self.column_filter)
        df_binned = apply_bins(df_filtered, self.binning)
        #treating missing values as categories 
        for col in df_binned.columns:
            if str(df_binned[col].dtype) == "category":
                df_binned[col] = df_binned[col].cat.add_categories(["MISSING"])
        df_binned = df_binned.fillna("MISSING")
        

        #hint2 : iteration and calculation of the relative frequency observed 
        predictions = pd.DataFrame(index=df_binned.index, columns=self.labels)

        features = [col for col in df_binned.columns if col not in ['CLASS', 'ID']]

        for idx in df_binned.index:
            row = df_binned.loc[idx]

            # storing non normalized probabilities 
            class_probabilities = {}

            for label in self.labels:

                # Hint 3
                prob = self.class_priors[label]

                # For each feature, we calculate P(feature_value | class)
                for feature in features:
                    feature_value = row[feature]

                    if pd.notna(feature_value):
                        count_value_and_class = self.feature_class_value_counts[feature].get(
                            (feature_value, label), 0
                        )
                        count_class = self.feature_class_counts[feature].get(label, 0)

                        # P(feature_value | label) = count(value,label) / count(label)
                        if count_class > 0:
                            relative_frequency = count_value_and_class / count_class
                            prob *= relative_frequency
                        else:
                            prob *= 0

                # Store non-normalized probability
                class_probabilities[label] = prob

            # Hint 4 now we will normalize probabilities 
            total_prob = sum(class_probabilities.values())

            if total_prob > 0:
                for label in self.labels:
                    predictions.loc[idx, label] = class_probabilities[label] / total_prob
            else:    
                for label in self.labels:
                    predictions.loc[idx, label] = self.class_priors[label]

        return predictions   




 

In [87]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

nb_model = NaiveBayes()

test_labels = glass_test_df["CLASS"]

nobins_values = [3,5,10]
bintype_values = ["equal-width","equal-size"]
parameters = [(nobins,bintype) for nobins in nobins_values for bintype in bintype_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    nb_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = nb_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values]),
                       columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

Training time (3, 'equal-width'): 0.05 s.
Testing time (3, 'equal-width'): 0.07 s.
Training time (3, 'equal-size'): 0.05 s.
Testing time (3, 'equal-size'): 0.06 s.
Training time (5, 'equal-width'): 0.04 s.
Testing time (5, 'equal-width'): 0.07 s.
Training time (5, 'equal-size'): 0.05 s.
Testing time (5, 'equal-size'): 0.06 s.
Training time (10, 'equal-width'): 0.05 s.
Testing time (10, 'equal-width'): 0.06 s.
Training time (10, 'equal-size'): 0.05 s.
Testing time (10, 'equal-size'): 0.06 s.



'results'

Accuracy  Brier score       AUC
3  equal-width  0.616822     0.622116  0.724335
   equal-size   0.607477     0.554782  0.780163
5  equal-width  0.644860     0.551101  0.771688
   equal-size   0.598131     0.581556  0.796675
10 equal-width  0.654206     0.527569  0.812887
   equal-size   0.588785     0.741668  0.751165

In [88]:
train_labels = glass_train_df["CLASS"]
nb_model.fit(glass_train_df)
predictions = nb_model.predict(glass_train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.8505
AUC on training set: 0.9687
Brier score on training set: 0.2263


### Comment on assumptions, things that do not work properly, etc.